***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [54]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME         PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
wikicluster  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [55]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [56]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from nltk.stem.snowball import SnowballStemmer
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  8 21:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [58]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [59]:
spark

In [60]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wikibucket208' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)


In [61]:
# Get some files from the bucket
def get_wiki_file_paths(all_wiki_files, num_files=3, all=False):
    # Dictionary to hold the file parts
    wiki_files_dict = {}

    # Regular expression to extract file number and part
    file_pattern = re.compile(r"multistream(\d+)_?(part\d+)?_preprocessed\.parquet")
    
    # Populate the dictionary with file numbers and their parts
    for file_path in all_wiki_files:
        match = file_pattern.search(file_path)
        if match:
            file_number = int(match.group(1))
            file_part = match.group(2) or "part1"  # Default to part1 if no part specified

            if file_number not in wiki_files_dict:
                wiki_files_dict[file_number] = []
            wiki_files_dict[file_number].append((file_part, file_path))

    # Sort the parts for each file number
    for file_number in wiki_files_dict:
        wiki_files_dict[file_number].sort()
    
    if(all):
        max_num = __builtin__.max(wiki_files_dict.keys())
        selected_files = list(sorted(wiki_files_dict.keys()))[:max_num]
    else:
        selected_files = list(sorted(wiki_files_dict.keys()))[:num_files]

    # Collect paths, ensuring all parts of a file number are included
    wiki_files_paths = []
    for file_number in selected_files:
        for _, file_path in wiki_files_dict[file_number]:
            wiki_files_paths.append(file_path)

    return wiki_files_paths


In [62]:
NUM_BUCKETS = 150
bucket_name = 'wikibucket208'
base_dir = f'{bucket_name}'

wiki_files_dir = f'{base_dir}/WikiFiles'

index_filename = f'{bucket_name}/InvertedIndex/'
inverted_dir = f'{base_dir}/InvertedIndex'
posting_locs_dir = f'{inverted_dir}/posting_locs'
pl_body_dir = f'{posting_locs_dir}/body'
pl_title_dir = f'{posting_locs_dir}/title'

In [63]:
all_files = True
num_wiki_files = 1   

wiki_files_paths = get_wiki_file_paths(paths,num_wiki_files, all_files)
parquetFile = spark.read.parquet(*wiki_files_paths)

In [64]:
# Count number of wiki pages
parquetFile.count()

6348910

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Part 1 - Building an inverted index

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [65]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [66]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/11 19:11:49 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [67]:
from inverted_index_gcp import InvertedIndex

Indexing Functions

In [68]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
# Word Count
# def word_count(text, id, stem_model='porter'):
#     tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
#     stemmed_tokens = stem_tokens(tokens, stem_model)
#     filtered_tokens = [token for token in stemmed_tokens if token not in all_stopwords]
    
#     tok_dic = Counter(filtered_tokens)
#     doc_length = len(filtered_tokens)  # Document length after stemming and removing stopwords
#     result = [(token, (id, count)) for token, count in tok_dic.items()]
#     return (id, (result, doc_length))
    

def word_count(text, id):

  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

  # YOUR CODE HERE
  stemmed_tokens = stem_tokens(tokens)

  tok_dic = {}
  non_stop_tokens = [token for token in tokens if token not in all_stopwords]
  for token in non_stop_tokens:
    tok_dic[token] = tok_dic.get(token, 0) + 1
  result = [(token, (id, tf)) for token, tf in tok_dic.items()]
  return result
    
    
# Reduce Word Count
def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])
    
    
# Calculate DF
def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))
    
    
# Write Partition
def partition_postings_and_write(postings, save_dir):
  def write_bucket_to_disk(bucket_id_and_postings, save_dir):
        # Pass both the tuple and the bucket_name to write_a_posting_list
    return  InvertedIndex.write_a_posting_list(bucket_id_and_postings, save_dir, bucket_name)


  postings_with_bucket_id = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))
  postings_grouped_by_bucket = postings_with_bucket_id.groupByKey().mapValues(list)
  bucket_ids = postings_grouped_by_bucket.map(
        lambda bucket_postings: write_bucket_to_disk((bucket_postings[0], list(bucket_postings[1])), save_dir)
  )
  return bucket_ids

In [69]:
# Tokenizer
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){,24}""", re.UNICODE)
def tokenize(text):
  return [token.group() for token in RE_WORD.finditer(text.lower())]

# Stop Words function
nltk_stop_words = set(stopwords.words('english'))

corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
all_stopwords = nltk_stop_words.union(corpus_stopwords)

def remove_stopwords(tokens):
    return [token for token in tokens if token not in all_stopwords]

In [70]:
# Stemmer Function
def stem_tokens(tokens, stem_model='porter'):
    if stem_model == 'porter':
        porter = PorterStemmer()
        stemmed_tokens = [porter.stem(token) for token in tokens]
    elif stem_model == 'snowball':
        snowball = SnowballStemmer(language='english')
        stemmed_tokens = [snowball.stem(token) for token in tokens]
    elif stem_model == None:
        return tokens
    else:
        raise ValueError("Unsupported stem model: {}".format(stem_model))
    return stemmed_tokens


# Stemmer for Postinglists


In [71]:
# Buckets so the data will be split efficiently

from collections import defaultdict
def _hash(s):
  return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

def tokens2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

<h2 style="color: blue;">Doc Length Calculation</h3>


In [72]:
def doc_length_mapper(text, doc_id):
    tokens = tokenize(text)
    stemmed = stem_tokens(tokens)
    non_sw_tokens = remove_stopwords(stemmed)
    doc_len = len(non_sw_tokens)
    
    bucket_id = tokens2bucket_id(str(doc_id))
    
    return(bucket_id, (doc_id, doc_len))

def doc_length_reducer(values):
    doc_lengths = {}
    total_length = 0
    doc_lengths = {doc_id: doc_len for doc_id, doc_len in values}
    total_length = __builtin__.sum(doc_lengths.values())
    doc_count = len(doc_lengths)
    avg_doc_len = total_length / doc_count if doc_count else 0

    return doc_lengths, total_length, doc_count, avg_doc_len

    
def write_to_gcp(save_dir, bucket_id, file_name, serialized_data):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob_name = f'{save_dir}/{bucket_id}{file_name}.pickle'
    blob = bucket.blob(blob_name)
    blob.upload_from_string(serialized_data)

    

In [73]:
type = 'body'
doc_text_pairs = parquetFile.select("text", "id").rdd

mapped_rdd = doc_text_pairs.map(lambda x: doc_length_mapper(x[0], x[1]))
grouped_rdd = mapped_rdd.groupByKey()
result_rdd = grouped_rdd.mapValues(doc_length_reducer)
results = result_rdd.collect()

avgdl = 0
doc_num = 0
tot_doc_len = 0

for bucket_id, (doc_lengths, total_length, doc_count, avg_doc_len) in results:
    doc_len_dir = f'InvertedIndex/doc_length/{type}'
    serialized_doc_lengths = pickle.dumps(doc_lengths)
    # Assuming you have a function to upload to cloud storage
    write_to_gcp(doc_len_dir, bucket_id, 'doc_len',serialized_doc_lengths)
    tot_doc_len += total_length
    doc_num += doc_count

bm25_calc = {}
bm25_calc['avgdl'] = tot_doc_len / doc_num if doc_num else 0
bm25_calc['N'] = doc_num
serialized_title_calcs = pickle.dumps(bm25_calc)
body_bm25 = f'InvertedIndex/bm25'
write_to_gcp(body_bm25, '', f'{type}_bm25', serialized_title_calcs)


    # ------------------------------------------------------------------------------------------------------------    

    
type = 'title'
doc_text_pairs = parquetFile.select("title", "id").rdd

mapped_rdd = doc_text_pairs.map(lambda x: doc_length_mapper(x[0], x[1]))
grouped_rdd = mapped_rdd.groupByKey()
result_rdd = grouped_rdd.mapValues(doc_length_reducer)
results = result_rdd.collect()

avgdl = 0
doc_num = 0
tot_doc_len = 0

for bucket_id, (doc_lengths, total_length, doc_count, avg_doc_len) in results:
    doc_len_dir = f'InvertedIndex/doc_length/{type}'
    serialized_doc_lengths = pickle.dumps(doc_lengths)
    # Assuming you have a function to upload to cloud storage
    write_to_gcp(doc_len_dir, bucket_id, 'doc_len',serialized_doc_lengths)
    tot_doc_len += total_length
    doc_num += doc_count

bm25_calc = {}
bm25_calc['avgdl'] = tot_doc_len / doc_num if doc_num else 0
bm25_calc['N'] = doc_num
serialized_title_calcs = pickle.dumps(bm25_calc)
title_bm25 = f'InvertedIndex/bm25'
write_to_gcp(title_bm25, '', f'{type}_bm25', serialized_title_calcs)